# ResNet headpose in Tensorflow for DeepLens

This notebook shows how to train an image classification model in Tensorflow on Amazon SageMaker and to prepare the trained model for AWS DeepLens deployment. 
The model used for this notebook is a RestNet model, trained with the headpose dataset.
See the following papers for more background:

[Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Dec 2015.

[Identity Mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Jul 2016.

The following scripts are modified from Amazon SageMaker sample, [ResNet CIFAR-10 with tensorboard](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk/tensorflow_resnet_cifar10_with_tensorboard)

### Set up the environment

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

s3_bucket = 'deeplens-sagemaker-0001'
headpose_folder = 'headpose'

#Bucket location to save your custom code in tar.gz format.
custom_code_folder = 'customTFcodes'
custom_code_upload_location = 's3://{}/{}/{}'.format(s3_bucket, headpose_folder, custom_code_folder)

#Bucket location where results of model training are saved.
model_artifacts_folder = 'TFartifacts'
model_artifacts_location = 's3://{}/{}/{}'.format(s3_bucket, headpose_folder, model_artifacts_folder)

#IAM execution role that gives SageMaker access to resources in your AWS account.
#We can use the SageMaker Python SDK to get the role from our notebook environment. 

role = get_execution_role()

## Create a training job using the sagemaker.TensorFlow estimator

### Complete source code
- [resnet_model_headpose.py](resnet_model_headpose.py): ResNet model
- [resnet_headpose.py](resnet_headpose.py): main script used for training and hosting

In [ ]:
from sagemaker.tensorflow import TensorFlow

source_dir = os.path.join(os.getcwd())

# AWS DeepLens currently supports TensorFlow version 1.4 (as of June 14th 2018). 
estimator = TensorFlow(entry_point='resnet_headpose.py',
                       framework_version = 1.4,
                       source_dir=source_dir,
                       role=role,
                       training_steps=25000, evaluation_steps=700,
                       train_instance_count=1, 
                       base_job_name='deeplens-TF-headpose',
                       output_path=model_artifacts_location,
                       code_location=custom_code_upload_location,
                       train_instance_type='ml.p2.xlarge',
                       train_max_run = 432000,
                       train_volume_size=100)


# Head-pose dataset "HeadPoseData_trn_test_x15_py2.pkl" is in the following S3 folder. 
dataset_location = 's3://{}/{}/datasets'.format(s3_bucket, headpose_folder)

estimator.fit(dataset_location)
# Enabling Tensorboard. 
#estimator.fit(dataset_location, run_tensorboard_locally=True)

## Make a frozen protobuff file (frozen_model.pb)
The trained model artifact needs to be converted to a frozen protobuff format, which is supported by AWS DeepLens' model optimizer.

### Fetch a trained model artifact (model.tar.gz) from S3
First, download model.tar.gz to the local folder. 

In [ ]:
import boto3
s3 = boto3.resource('s3')
key = '{}/{}/{}/output/model.tar.gz'.format(headpose_folder, model_artifacts_folder,estimator.latest_training_job.name)
print(key)
s3.Bucket(s3_bucket).download_file(key,'model.tar.gz')

You may find a model.tar.gz in your local directry. 

In [ ]:
!ls

### Untar the trained model artifact (model.tar.gz)

In [ ]:
!tar -xvf model.tar.gz

### Find the model directory

In [ ]:
import glob
model_dir = glob.glob('export/*/*')
# The model directory looks like 'export/Servo/{Assigned by Amazon SageMaker}'
print(model_dir)

### Freeze the graph and save it in the frozen protobuff format

In [ ]:
import tensorflow as tf
from tensorflow.python.tools import optimize_for_inference_lib
def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    
    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], model_dir)

        # We use a built-in TF helper to export variables to constants
        input_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        ) 

    # We generate the inference graph_def
    output_graph_def = optimize_for_inference_lib.optimize_for_inference(tf.graph_util.remove_training_nodes(input_graph_def),
                                                                         ['Const_1'], # an array of the input node(s)
                                                                         output_node_names.split(","), # an array of output nodes
                                                                         tf.float32.as_datatype_enum)
    # Finally we serialize and dump the output graph_def to the filesystem
    with tf.gfile.GFile('frozen_model.pb', "wb") as f:
            f.write(output_graph_def.SerializeToString())
    print("tf magic!")

In [ ]:
freeze_graph(model_dir[0], 'softmax_tensor')

It should be noted that we knew the names of input and output nodes (i.e. Const_1 and softmax_tensor) by examining TensorBoard in advance. 

You may find a frozen_model.pb in your local directry. Now you are ready to deploy the file to AWS DeepLens.

In [ ]:
!ls 

### Put frozen_model.pb back to S3

In [ ]:
data = open('frozen_model.pb', "rb")
key = '{}/{}/{}/output/frozen_model.pb'.format(headpose_folder, model_artifacts_folder,estimator.latest_training_job.name)
s3.Bucket(s3_bucket).put_object(Key=key, Body=data)

In [ ]:
print('s3://{}/{}'.format(s3_bucket, key))

# (Extra) Deploy the trained model to SageMaker Endpoint 

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Make a prediction with fake data to verify the endpoint is up

In [ ]:
import numpy as np

random_image_data = np.random.rand(1,84, 84, 3)
predictor.predict(random_image_data)

## Make a prediction with a headpose image.

In [ ]:
import cv2
from PIL import Image
import numpy as np
import boto3
import os
import matplotlib.pyplot as plt
%matplotlib inline
import urllib

role = get_execution_role()

sample_ims_location = 'https://s3.amazonaws.com/deeplens-sagemaker-0001/headpose/testIMs/IMG_1242.jpeg'

print(sample_ims_location)

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.urlretrieve(url, filename)
    print(filename)
    return cv2.imread(filename).astype(np.float32)
        
im_true = download(sample_ims_location)

im = im_true.astype(np.float32)/255.0 # Normalized

crop_uly = 62
crop_height = 360
crop_ulx = 100
crop_width = 360

im = im[crop_uly:crop_uly + crop_height, crop_ulx:crop_ulx + crop_width]
im_crop = im
plt.imshow(im_crop[:,:,::-1])
plt.show()

im = cv2.resize(im, (84, 84))

plt.imshow(im[:,:,::-1])
plt.show()

print(im.shape)
im = np.expand_dims(im, axis=0)
print(im.shape)

In [ ]:
predictor.predict(im)

# Cleaning up
To avoid incurring charges to your AWS account for the resources used in this tutorial you need to delete the **SageMaker Endpoint:**

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)